In [1]:
import tempfile
import subprocess
from tqdm import tqdm
from time import sleep
from tiresias.server import api

In [2]:
clients = []
nb_clients = 101
server = subprocess.Popen(['tiresias-server'])
for client_id in tqdm(range(nb_clients)):
    clients.append(subprocess.Popen(['tiresias-client', "--db_dir", tempfile.mkdtemp(), "--db_port", str(8000 + client_id)]))
    sleep(0.5)

100%|██████████| 101/101 [00:51<00:00,  1.97it/s]


In [3]:
import pandas as pd
results = []

for sample_size in tqdm([10, 20, 30, 40, 50, 60, 70, 80, 90, 100]):
    query_id = api.create_query("http://localhost:3000/", {
        "type": "basic",
        "epsilon": 1.0,
        "min_sample_size": sample_size,
        "featurizer": "SELECT x1 FROM hello_world.two_sum",
        "aggregator": "median"
    })
    sleep(5)
    query = api.fetch_query("http://localhost:3000/", query_id)
    results.append({
        "task": "median",
        "nb_users": sample_size,
        "running_time": query["end"] - query["start"]
    })

    query_id = api.create_query("http://localhost:3000/", {
        "type": "machine_learning",
        "epsilon": 10.0,
        "min_sample_size": sample_size,
        "featurizer": "SELECT x1, x2, y FROM hello_world.two_sum",
        "aggregator": {
            "model": "LinearRegression",
            "inputs": ["x1", "x2"],
            "output": "y",
            "data_norm": [(0.0, 100.0), (0.0, 100.0)]
        }
    })
    sleep(5)
    query = api.fetch_query("http://localhost:3000/", query_id)
    results.append({
        "task": "regression",
        "nb_users": sample_size,
        "running_time": query["end"] - query["start"]
    })
    
pd.DataFrame(results)

100%|██████████| 10/10 [01:40<00:00, 10.03s/it]


,nb_users,running_time,task
0,10,2.351218,median
1,10,2.496199,regression
2,20,2.343990,median
3,20,2.292343,regression
4,30,2.002056,median
5,30,2.068459,regression
6,40,4.645695,median
7,40,2.017239,regression
8,50,4.940688,median
9,50,2.928759,regression


In [4]:
df = pd.DataFrame(results)
df[df["task"]=="median"].to_csv("median.csv", index=False)
df[df["task"]=="regression"].to_csv("regression.csv", index=False)
df

,nb_users,running_time,task
0,10,2.351218,median
1,10,2.496199,regression
2,20,2.343990,median
3,20,2.292343,regression
4,30,2.002056,median
5,30,2.068459,regression
6,40,4.645695,median
7,40,2.017239,regression
8,50,4.940688,median
9,50,2.928759,regression
